# Manejo de Archivos en Python

Este notebook demuestra cómo leer y escribir diferentes formatos de archivos: CSV, Parquet, JSON, Excel y más.

**Referencia:** [Manejo de Archivos](../fundamentos/manejo-de-archivos.md)

**Objetivos:**
- Leer y escribir CSV
- Trabajar con Parquet (formato optimizado para analytics)
- Convertir entre formatos
- Comparar rendimiento y tamaños
- Procesar archivos grandes con chunks

## 1. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import json

print("✅ Librerías importadas")

## 2. Leer CSV

In [ ]:
# Leer CSV básico
df_csv = pd.read_csv('../data/ventas.csv')
print(f"✅ CSV cargado: {len(df_csv)} registros")
print(f"Columnas: {df_csv.columns.tolist()}")
df_csv.head()

# Leer CSV con opciones
print("\n=== LEER CSV CON OPCIONES ===")
df_csv_opciones = pd.read_csv(
    '../data/ventas.csv',
    usecols=['producto', 'precio', 'total'],  # Solo estas columnas
    dtype={'precio': 'float32'}  # Tipo específico
)
print(f"Columnas seleccionadas: {df_csv_opciones.columns.tolist()}")
print(f"Tipo de precio: {df_csv_opciones['precio'].dtype}")
df_csv_opciones.head()

## 3. Leer y escribir Parquet

In [ ]:
# Leer Parquet
start = time.time()
df_parquet = pd.read_parquet('../data/ventas.parquet')
time_parquet = time.time() - start

print(f"✅ Parquet cargado: {len(df_parquet)} registros en {time_parquet:.4f}s")
df_parquet.head()

# Comparar velocidad con CSV
start = time.time()
df_csv_test = pd.read_csv('../data/ventas.csv')
time_csv = time.time() - start

print(f"\n📊 COMPARACIÓN DE VELOCIDAD:")
print(f"CSV: {time_csv:.4f}s")
print(f"Parquet: {time_parquet:.4f}s")
if time_parquet > 0:
    print(f"Parquet es {time_csv/time_parquet:.2f}x más rápido")

In [ ]:
# Escribir Parquet
df_csv = pd.read_csv('../data/ventas.csv')

# Crear directorio de salida si no existe
os.makedirs('../data/output', exist_ok=True)

# Convertir a Parquet con opciones
df_csv.to_parquet(
    '../data/output/ventas_convertido.parquet',
    engine='pyarrow',
    compression='snappy',
    index=False
)

print("✅ Parquet guardado exitosamente en ../data/output/")

# Comparar tamaños
size_csv = os.path.getsize('../data/ventas.csv')
size_parquet = os.path.getsize('../data/ventas.parquet')
size_parquet_conv = os.path.getsize('../data/output/ventas_convertido.parquet')

print(f"\n📊 COMPARACIÓN DE TAMAÑOS:")
print(f"CSV: {size_csv:,} bytes ({size_csv/1024:.2f} KB)")
print(f"Parquet original: {size_parquet:,} bytes ({size_parquet/1024:.2f} KB)")
print(f"Parquet convertido: {size_parquet_conv:,} bytes ({size_parquet_conv/1024:.2f} KB)")
if size_parquet > 0:
    print(f"Parquet es {size_csv/size_parquet:.2f}x más pequeño que CSV")

## 4. Leer Parquet con opciones avanzadas

In [ ]:
# Leer solo columnas específicas (más eficiente)
df_parquet_cols = pd.read_parquet(
    '../data/ventas.parquet',
    columns=['producto', 'precio', 'total']
)

print("=== PARQUET CON COLUMNAS ESPECÍFICAS ===")
print(f"Columnas: {df_parquet_cols.columns.tolist()}")
print(f"Registros: {len(df_parquet_cols)}")
df_parquet_cols.head()

# Nota: Los filtros pushdown requieren que la columna esté en el formato correcto
# En este ejemplo, podríamos filtrar por fecha si fuera necesario
print("\n💡 Tip: Parquet permite filtros pushdown para leer solo datos necesarios")

## 5. Trabajar con JSON

In [ ]:
# Leer CSV y convertir a JSON
df = pd.read_csv('../data/ventas.csv')

# Crear directorio de salida si no existe
os.makedirs('../data/output', exist_ok=True)

# Escribir JSON con diferentes orientaciones
df.to_json('../data/output/ventas.json', orient='records', index=False)
print("✅ JSON guardado en ../data/output/ (orient='records')")

# Leer JSON
df_json = pd.read_json('../data/output/ventas.json')
print(f"\n✅ JSON cargado: {len(df_json)} registros")
df_json.head()

# Comparar tamaños
size_json = os.path.getsize('../data/output/ventas.json')
print(f"\n📊 Tamaño JSON: {size_json:,} bytes ({size_json/1024:.2f} KB)")

## 6. Procesar archivos grandes (simulado con chunks)

In [ ]:
# Simular procesamiento de archivo grande usando chunks
# En este ejemplo, procesamos el CSV en chunks pequeños para demostrar la técnica

chunk_size = 10  # Chunks pequeños para el ejemplo
chunks_procesados = []

print("=== PROCESAR CSV EN CHUNKS ===")
for i, chunk in enumerate(pd.read_csv('../data/ventas.csv', chunksize=chunk_size)):
    # Procesar cada chunk: filtrar ventas con total > 500
    chunk_filtrado = chunk[chunk['total'] > 500]
    chunks_procesados.append(chunk_filtrado)
    print(f"Chunk {i+1}: {len(chunk)} registros → {len(chunk_filtrado)} filtrados")

# Combinar resultados
df_final = pd.concat(chunks_procesados, ignore_index=True)
print(f"\n✅ Total de registros después de procesar: {len(df_final)}")
print(f"Registros con total > 500: {len(df_final)}")
df_final.head()

# Crear directorio de salida si no existe
os.makedirs('../data/output', exist_ok=True)

# Guardar resultado en Parquet
df_final.to_parquet('../data/output/ventas_filtradas.parquet', index=False)
print("\n✅ Resultado guardado en ../data/output/ventas_filtradas.parquet")

## 7. Resumen: Comparación de formatos

In [ ]:
print("=" * 70)
print("RESUMEN: COMPARACIÓN DE FORMATOS")
print("=" * 70)

# Comparar tamaños
archivos = {
    'CSV': '../data/ventas.csv',
    'Parquet': '../data/ventas.parquet',
    'JSON': '../data/output/ventas.json'  # Archivo generado en este notebook
}

print("\n📊 TAMAÑOS DE ARCHIVOS:")
for formato, ruta in archivos.items():
    if os.path.exists(ruta):
        size = os.path.getsize(ruta)
        print(f"{formato:10s}: {size:>8,} bytes ({size/1024:>6.2f} KB)")

# Comparar velocidades de lectura
print("\n⚡ VELOCIDADES DE LECTURA:")
for formato, ruta in archivos.items():
    if os.path.exists(ruta):
        start = time.time()
        if formato == 'CSV':
            pd.read_csv(ruta)
        elif formato == 'Parquet':
            pd.read_parquet(ruta)
        elif formato == 'JSON':
            pd.read_json(ruta)
        tiempo = time.time() - start
        print(f"{formato:10s}: {tiempo:>8.4f}s")

print("\n💡 RECOMENDACIONES:")
print("✅ CSV: Para intercambio de datos, lectura humana")
print("✅ Parquet: Para datos procesados, analytics, pipelines")
print("✅ JSON: Para APIs, integraciones web")
print("=" * 70)